This is For Webscraping Real time Stock data from Yahoo Finance and store it locally.

In [1]:
import pandas as pd
import datetime
import requests
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup


In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/114.0'}

In [46]:
def web_content_div(web_content, class_path, value):
    web_content_div = web_content.find_all('div', {'class':class_path})
    try:
        if value != "None":
            spans =  web_content_div[0].find_all(value)
            texts = [span.get_text() for span in spans]
        else:
            texts =web_content_div[0].get_text("|", strip=True)
            texts = texts.split("|") 
        #spans =  web_content_div[0].find_all('fin-streamer')
        
    except IndexError:
        texts = []
    return texts


def real_time_price(stock_code):
    Error = 0
    url = "https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch".format(stock_code,stock_code)
    try:
        r = requests.get(url,headers=headers)
        web_content = BeautifulSoup(r.text,"lxml")
        
        # First getting the Price and Price change
        texts = web_content_div(web_content,'My(6px) Pos(r) smartphone_Mt(6px) W(100%)','fin-streamer')
        print("texts received")
        print(texts)
        price, change, volume, latest_pattern, one_year_target, = [],[], [],[], []

        if texts != []:
            price, change,  = texts[0], texts[1] +' '+ texts[2]
        
        else:
            price, change= [], []


        # Second getting the Volume 
        texts = web_content_div(web_content,'D(ib) W(1/2) Bxz(bb) Pend(12px) Va(t) ie-7_D(i) smartphone_D(b) smartphone_W(100%) smartphone_Pend(0px) smartphone_BdY smartphone_Bdc($seperatorColor)','fin-streamer')
        print("volume received")
        
        if texts != []:
            volume = texts[0]
            print(volume)
        else:
            volume = []

        #Now getting the 1 year est
        texts = web_content_div(web_content,'D(ib) W(1/2) Bxz(bb) Pstart(12px) Va(t) ie-7_D(i) ie-7_Pos(a) smartphone_D(b) smartphone_W(100%) smartphone_Pstart(0px) smartphone_BdB smartphone_Bdc($seperatorColor)','None')
    
        if texts != []:
            print("one year tar")

            market_cap = texts[1]
            PE_ratio_TTM = texts[5]
            EPS_TTM = texts[7]
            earning_date = texts[9]+texts[10]+texts[11]
            one_year_target = texts[-1]
            print(one_year_target)
            
            
        else:
            one_year_target = []
            market_cap = []
            PE_ratio_TTM = []
            EPS_TTM = []
            earning_date = []
                
    except ConnectionError:
        price, change, volume, latest_pattern, one_year_target = [],[], [],[], []
        Error = 1
        print("Connection Error")
    
    return price, change, volume, latest_pattern, one_year_target, Error

In [25]:
"""stock_code = 'AAPL'
url = "https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch".format(stock_code,stock_code)
r = requests.get(url,headers=headers)
web_content = BeautifulSoup(r.text,"lxml")
texts = web_content_div(web_content,'D(ib) W(1/2) Bxz(bb) Pstart(12px) Va(t) ie-7_D(i) ie-7_Pos(a) smartphone_D(b) smartphone_W(100%) smartphone_Pstart(0px) smartphone_BdB smartphone_Bdc($seperatorColor)','None')
print("text is")
print(texts)"""

text is
[]


In [42]:
Stock = ['AAPL', ]
print(real_time_price(Stock))

texts received
[]
volume received
([], [], [], [], [], 0)


In [49]:
while(True):
    info = []
    # Shift to US time 13 hours winter, 12 hours summer
    time_stamp = datetime.datetime.now() - datetime.timedelta(hours=13)
    time_stamp = time_stamp.strftime("%Y-%m-%d %H:%M:%S")

    for stock_code in Stock:
        stock_price, change, volume,latest_pattern, one_year_target, Error = real_time_price(stock_code)
        info.append(stock_price)
        info.extend([change])
        info.extend([volume])
        info.extend([one_year_target])
        

    if Error != 0:
        print("Connection Error")
        break
    
    col = [time_stamp]
    col.extend(info)
    df = pd.DataFrame(col)
    df = df.T
    path = "C:\\Users\\User\\Desktop\\QuantPack\\"
    path += str(time_stamp[0:11])+ 'stock_data.csv'

    df.to_csv(path, mode='a', header=False)

texts received
['192.30', '-1.12', '(-0.58%)', '', '']
volume received
32,339,415
one year tar
198.01
texts received
['192.29', '-1.12', '(-0.58%)', '', '']
volume received
32,346,098
one year tar
198.01


KeyboardInterrupt: 